In [9]:
from VisionEngine.utils.config import process_config
from VisionEngine.utils import factory
import sys

import tensorflow as tf

In [2]:
config_file = '../VisionEngine/configs/guppy_vae_config.json'
config = process_config(config_file)

In [3]:
data_loader = factory.create(
            "VisionEngine.data_loaders."+config.data_loader.name
            )(config)

importing VisionEngine.data_loaders.guppy_vae_data_loader
getattr GuppyDataLoader


In [4]:
model = factory.create(
            "VisionEngine.models."+config.model.name
            )(config)

importing VisionEngine.models.vae_model
getattr VAEModel
Model: "vlae"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 984064), (No 9918848     input_2[0][0]                    
__________________________________________________________________________________________________
normal_variational (NormalVaria (None, 10)           19681301    encoder[1][0]                    
__________________________________________________________________________________________________
normal_variational_1 (NormalVar (None, 10)           4014101     encoder[1][1]                    
______________________________________

In [5]:
checkpoint_path = '/home/etheredge/Workspace/VisionEngine/VisionEngine/models/checkpoints/guppy_vae/2020-056-15/guppy_vae-304-684.91.hdf5'

In [6]:
model.load(checkpoint_path)

Loading model checkpoint /home/etheredge/Workspace/VisionEngine/VisionEngine/models/checkpoints/guppy_vae/2020-056-15/guppy_vae-304-684.91.hdf5 ...

Model loaded


In [34]:
def sample_likelihood(x, x_hat):
    mse = - tf.losses.mean_squared_error(Flatten()(x), Flatten()(x_hat))
    return tf.log(1./(tf.sqrt(2.*math.pi))*tf.exp(-.5*(mse)**2.))

def embed_images(x):
    x = model.model.get_layer('encoder').predict(x, batch_size=10)
    return [model.model.get_layer('normal_variational')(x[0]), 
            model.model.get_layer('normal_variational_1')(x[1]),
            model.model.get_layer('normal_variational_2')(x[2]),
            model.model.get_layer('normal_variational_3')(x[3])]

def reconstruct_images(z):
    return model.model.get_layer('decoder').predict(z)

In [35]:
with tf.device("GPU:0"):
    z = embed_images(data_loader.get_test_data()[0])
    x_hat = reconstruct_images(z)
    likelihood = sample_likelihood(data_loader.get_test_data, x_hat)

NameError: name 'Flatten' is not defined

In [33]:
for layer in model.model.layers:
    print(layer.name)

input_2
encoder
normal_variational
normal_variational_1
normal_variational_2
normal_variational_3
decoder
